## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
%matplotlib inline

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lalmohan,BD,22.3384,90.7419,84.90,76,100,9.62,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,71.94,74,100,17.96,light rain
2,2,Kapaa,US,22.0752,-159.3190,79.12,83,75,14.97,broken clouds
3,3,Sayyan,YE,15.1718,44.3244,63.43,68,96,5.26,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,34.48,97,52,3.06,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,71.94,74,100,17.96,light rain
2,2,Kapaa,US,22.0752,-159.3190,79.12,83,75,14.97,broken clouds
6,6,Kununurra,AU,-15.7667,128.7333,69.80,25,100,4.47,overcast clouds
9,9,Saint-Augustin,CA,51.2260,-58.6502,72.79,60,45,15.93,scattered clouds
10,10,Aguimes,ES,27.9054,-15.4461,77.63,65,20,31.07,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                243
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

243


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,71.94,74,100,17.96,light rain
2,2,Kapaa,US,22.0752,-159.3190,79.12,83,75,14.97,broken clouds
6,6,Kununurra,AU,-15.7667,128.7333,69.80,25,100,4.47,overcast clouds
9,9,Saint-Augustin,CA,51.2260,-58.6502,72.79,60,45,15.93,scattered clouds
10,10,Aguimes,ES,27.9054,-15.4461,77.63,65,20,31.07,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,71.94,light rain,-23.1203,-134.9692,
2,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,
6,Kununurra,AU,69.80,overcast clouds,-15.7667,128.7333,
9,Saint-Augustin,CA,72.79,scattered clouds,51.2260,-58.6502,
10,Aguimes,ES,77.63,few clouds,27.9054,-15.4461,
12,Isangel,VU,69.91,overcast clouds,-19.5500,169.2667,
13,Akcakoca,TR,77.76,clear sky,41.0866,31.1162,
14,Kamaishi,JP,75.81,few clouds,39.2667,141.8833,
20,Altay,CN,69.33,overcast clouds,47.8667,88.1167,
22,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,71.94,light rain,-23.1203,-134.9692,People ThankYou
2,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Kununurra,AU,69.80,overcast clouds,-15.7667,128.7333,Hotel Kununurra
9,Saint-Augustin,CA,72.79,scattered clouds,51.2260,-58.6502,
10,Aguimes,ES,77.63,few clouds,27.9054,-15.4461,Hotel Villa de Aguimes


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,71.94,light rain,-23.1203,-134.9692,People ThankYou
2,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Kununurra,AU,69.80,overcast clouds,-15.7667,128.7333,Hotel Kununurra
9,Saint-Augustin,CA,72.79,scattered clouds,51.2260,-58.6502,
10,Aguimes,ES,77.63,few clouds,27.9054,-15.4461,Hotel Villa de Aguimes
12,Isangel,VU,69.91,overcast clouds,-19.5500,169.2667,Tanna Lodge
13,Akcakoca,TR,77.76,clear sky,41.0866,31.1162,Turkuaz Beach Otel
14,Kamaishi,JP,75.81,few clouds,39.2667,141.8833,Hotel Route Inn Kamaishi
20,Altay,CN,69.33,overcast clouds,47.8667,88.1167,Jinqiao Hotel
22,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))